In [1]:
import os
import numpy as np
import math
import pickle
import natsort
from probreg import cpd, gmmtree, filterreg, bcpd, math_utils
import copy
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_cell_loc_from_df(HE_quant_df, MxIF_quant_df):
    he_x = HE_quant_df["Centroid X µm"]
    he_y = HE_quant_df["Centroid Y µm"]
    mxif_x = MxIF_quant_df["Centroid X µm"]
    mxif_y = MxIF_quant_df["Centroid Y µm"]

    source = np.array([he_x, he_y]).T
    target = np.array([mxif_x, mxif_y]).T
    return source, target

def align_cell_segmentation(source, target):
    # compute cpd registration
    # set update_scale to False, as the cell locations were denoted with microns (calculated based on pixel size)
    # TODO: But how to apply the affine transformation obtained from this registration if the pixel sizes are different?
    # TODO: sx=sy=source_pixel_size/target_pixel_size
    tf_param, sigma2, q = cpd.registration_cpd(source, target, maxiter=150, update_scale=False)

    result = copy.deepcopy(source)
    r_points = tf_param.transform(result)

    return tf_param, r_points, sigma2, q

def get_TMA_core_list(img_path: str) -> list:
    img_fn_list = os.listdir(img_path)
    roi_list = [i.split("_")[0] for i in img_fn_list]
    return list(set(roi_list))

In [17]:
HE_pxiel_size = 0.2201  # unit micron
MxIF_pixel_size = 0.325

pix_scale = HE_pxiel_size / MxIF_pixel_size
target_pixel_size = MxIF_pixel_size


def save_CPD_trans(MxIF_export_dir, HE_export_dir, cpd_output_dir):
    ROI_list = natsort.natsorted(get_TMA_core_list(MxIF_export_dir))
    for roi_id in ROI_list:
        print("\t Processing: %s" % roi_id)
        trans_fn = os.path.join(cpd_output_dir, roi_id + "_trans.dat")
        if os.path.exists(trans_fn):
            print("\t\t File already exist")
        else:    
            HE_quant_fn = os.path.join(HE_export_dir, roi_id + "_" + seg + "_QUANT.tsv")
            MxIF_quant_fn = os.path.join(MxIF_export_dir, roi_id + "_" + seg + "_QUANT.tsv")
            HE_quant_df = pd.read_csv(HE_quant_fn, sep='\t')
            MxIF_quant_df = pd.read_csv(MxIF_quant_fn, sep='\t')
            source, target = get_cell_loc_from_df(HE_quant_df, MxIF_quant_df)  # without cell density
            print("\t\t Number of points: (%d, %d)" % (len(source), len(target)))

            tf_param, r_points, sigma2, q = align_cell_segmentation(source, target)

            R = tf_param.rot
            T = tf_param.t
            M = np.array([[pix_scale * R[0, 0], pix_scale * R[0, 1], T[0] / target_pixel_size],
                          [pix_scale * R[1, 0], pix_scale * R[1, 1], T[1] / target_pixel_size]]).astype(float)

            theta = math.atan(R[1, 0]/R[0, 0])
            degrees = theta * ( 180.0 / math.pi )
            s = tf_param.scale
            delta = math.sqrt(M[0, 2]**2 + M[1, 2]**2)

            data = [theta, degrees, s, delta, M]
            with open(trans_fn, "wb") as f:
                pickle.dump(data, f)


In [18]:
import platform
if "windows" in platform.system().lower():
    data_root_dir = r"\\mfad\researchmn\HCPR\HCPR-GYNECOLOGICALTUMORMICROENVIRONMENT\Archive\WSIs\Ovarian_TMA\AlignmentEval"
elif "linux" in platform.system().lower():
    data_root_dir = "/infodev1/non-phi-data/junjiang/Ovarian_TMA/AlignmentEval"
else:
    raise Exception("platform not defined")

In [ ]:
sec_list = ["Sec1", "Sec2"]
seg_method_list = ["StarDist", "Watershed"]

for sec in sec_list:
    if sec == "Sec1":
        for seg in seg_method_list:
            print("Running %s for %s" % (sec, seg))
            if seg == "StarDist":
                HE_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_HE_Sec1", "export")
                MxIF_export_dir =os.path.join(data_root_dir, "QuPathAnnoProj_MxIF", "export")
                cpd_output_dir = os.path.join(data_root_dir, "Sec1_stardist_CPD")
            elif seg == "Watershed":
                HE_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_HE_Sec1_watershed", "export")
                MxIF_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_MxIF_watershed", "export")
                cpd_output_dir = os.path.join(data_root_dir, "Sec1_watershed_CPD")
            else:
                raise Exception("Unknown segmentation method")
            
            save_CPD_trans(MxIF_export_dir, HE_export_dir, cpd_output_dir)
    elif sec == "Sec2":
        for seg in seg_method_list:
            print("Running %s for %s" % (sec, seg))
            if seg == "StarDist":
                HE_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_HE_Sec2", "export")
                MxIF_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_MxIF", "export")
                cpd_output_dir = os.path.join(data_root_dir, "Sec2_stardist_CPD")
            elif seg == "Watershed":
                HE_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_HE_Sec2_watershed", "export")
                MxIF_export_dir = os.path.join(data_root_dir, "QuPathAnnoProj_MxIF_watershed", "export")
                cpd_output_dir = os.path.join(data_root_dir, "Sec2_watershed_CPD")
            else:
                raise Exception("Unknown segmentation method")

            save_CPD_trans(MxIF_export_dir, HE_export_dir, cpd_output_dir)
    else:
        raise Exception("Unknown section name")




Running Sec1 for StarDist
processing: A-8
File already exist
processing: A-22
File already exist
processing: B-9
File already exist
processing: B-11
File already exist
processing: B-12
File already exist
processing: B-15
File already exist
processing: B-21
File already exist
processing: C-10
File already exist
processing: C-15
File already exist
processing: C-21
File already exist
processing: C-24
File already exist
processing: D-6
File already exist
processing: E-13
File already exist
processing: F-10
File already exist
processing: G-7
File already exist
processing: H-13
File already exist
processing: H-16
File already exist
processing: H-20
File already exist
processing: I-15
File already exist
processing: I-17
Number of points: (3659, 4958)
Running Sec1 for Watershed
processing: A-8
Number of points: (3737, 2917)
processing: A-22
Number of points: (2261, 1900)
processing: B-9
Number of points: (5135, 4135)
processing: B-11
Number of points: (3586, 2794)
processing: B-12
Number of po